In [1]:
monitor_resp = [
    {
        "run_id": "8042932c-101a-4ddc-a808-00c092797960",
        "algorithm_used": "XGBClassifier",
        "status": "Successful"
    },
    {
        "run_id": "16e3f179-bb70-4632-9a39-9a6fac4a5581",
        "algorithm_used": "DecisionTreeClassifier",
        "status": "Successful"
    },
    {
        "run_id": "ef47a0a5-1a74-4a14-b0fd-8dbbd322057f",
        "algorithm_used": "GradientBoostingClassifier",
        "status": "Running"
    },
    {
        "run_id": "26a15f9c-29fd-4d55-82fb-3d38cc18e4b8",
        "algorithm_used": "RandomForestClassifier",
        "status": "Running"
    }
]

In [4]:
from fosforio.manager import get_conn_details_from_ds_name
from snowflake.snowpark.session import Session
from snowflake.ml.registry.registry import Registry

In [2]:
def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [3]:
session = get_session("EMPLOYEE_10L","0e0fb803-22db-4d88-9f2f-f6f75b6abcf0")

In [5]:
reg = Registry(session=session)

In [33]:
%%time
model_list = session.sql("show models like '%exp6789%'").collect()

CPU times: user 27.9 ms, sys: 0 ns, total: 27.9 ms
Wall time: 104 ms


In [34]:
model_list

[Row(created_on=datetime.datetime(2024, 7, 7, 20, 4, 19, 969000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), name='MY_RUN_LISTING_EXP_TEST12345_EXP6789_16E3F179_BB70_4632_9A39_9A6FAC4A5581', database_name='FIRST_DB', schema_name='PUBLIC', comment=None, owner='ADITYASINGH', default_version_name='V1', versions='["V1"]'),
 Row(created_on=datetime.datetime(2024, 7, 7, 20, 11, 8, 496000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), name='MY_RUN_LISTING_EXP_TEST12345_EXP6789_8042932C_101A_4DDC_A808_00C092797960', database_name='FIRST_DB', schema_name='PUBLIC', comment=None, owner='ADITYASINGH', default_version_name='V1', versions='["V1"]')]

In [39]:
for model in model_list:
    model_name = model["name"]
    model_run_id = model_name[-36:].replace('_','-').lower()
    model_obj = reg.get_model(model_name)
    for run in monitor_resp:
        if run.get('run_id') == model_run_id:
            run.update({f"model_metric: {json.dumps(model_obj.default.get_metric('model_metrics'))}"})
print(monitor_resp)

ValueError: dictionary update sequence element #0 has length 211; 2 is required

In [29]:
%%time
m = reg.get_model("MY_RUN_LISTING_EXP_TEST12345_EXP6789_16E3F179_BB70_4632_9A39_9A6FAC4A5581")

CPU times: user 68.1 ms, sys: 735 µs, total: 68.8 ms
Wall time: 1.29 s


In [18]:
dir(m)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_model_name',
 '_model_ops',
 '_ref',
 'comment',
 'default',
 'delete_version',
 'description',
 'fully_qualified_name',
 'get_tag',
 'name',
 'rename',
 'set_tag',
 'show_tags',
 'show_versions',
 'unset_tag',
 'version',
 'versions']

In [30]:
m.show_tags()

{'FIRST_DB.PUBLIC.ACCURACY_SCORE': '0.92473',
 'FIRST_DB.PUBLIC.ALGORITHMS': '[{"algorithm_name": "snowflake.ml.modeling.tree.DecisionTreeClassifier"}]',
 'FIRST_DB.PUBLIC.ALGORITHM_CATEGORY': 'Snowparkml',
 'FIRST_DB.PUBLIC.ALGORITHM_TYPE': 'Classification',
 'FIRST_DB.PUBLIC.DATASET_NAME': 'EMPLOYEE_10L',
 'FIRST_DB.PUBLIC.EXPERIMENT_ID': 'exp6789',
 'FIRST_DB.PUBLIC.EXPERIMENT_NAME': 'My_run_listing_exp',
 'FIRST_DB.PUBLIC.F1_SCORE': '0.8841251148525358',
 'FIRST_DB.PUBLIC.LOG_LOSS': '2.7149916732535027',
 'FIRST_DB.PUBLIC.MONITOR_RUN_ID': '16e3f179-bb70-4632-9a39-9a6fac4a5581',
 'FIRST_DB.PUBLIC.PRECISION_SCORE': '0.9379067021773153',
 'FIRST_DB.PUBLIC.PROJECT_ID': 'test12345',
 'FIRST_DB.PUBLIC.RECALL_SCORE': '0.8361769313619897',
 'FIRST_DB.PUBLIC.ROC_AUC': '0.9036114375207266',
 'FIRST_DB.PUBLIC.STORED_PROCEDURE': 'run_experiment',
 'FIRST_DB.PUBLIC.TARGET_COLUMN': 'LEAVEORNOT'}

In [31]:
m.default.get_metric('model_metrics')

{'accuracy_score': 0.92473,
 'f1_score': 0.8841251148525358,
 'recall_score': 0.8361769313619897,
 'precision_score': 0.9379067021773153,
 'roc_auc': 0.9036114375207266,
 'log_loss': 2.7149916732535027}